# Task 3: Forecast Future Market Trends (TSLA)

## Objective
Use the best-performing model from Task 2 to forecast Tesla's future stock prices for 6-12 months and analyze the results for actionable insights.

## Deliverables
- Forecast visualization with confidence intervals
- Trend analysis summary (1-2 paragraphs)
- List of identified opportunities and risks
- Critical assessment of forecast reliability over different time horizons

## 1. Imports and Setup

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os

from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.float_format', '{:.2f}'.format)

print('Setup complete!')

Setup complete!


## 2. Load Data and Model Info

In [2]:
# Load TSLA prices
prices = pd.read_csv('../data/processed/adj_close_prices.csv', parse_dates=['Date'], index_col='Date')
prices = prices.sort_index()
tsla = prices['TSLA'].dropna().astype(float)

print(f'TSLA data: {tsla.index.min().date()} to {tsla.index.max().date()}')
print(f'Total observations: {len(tsla)}')
print(f'Last price: ${tsla.iloc[-1]:.2f}')

TSLA data: 2015-01-02 to 2026-01-14
Total observations: 2775
Last price: $74.43


In [3]:
# Load ARIMA info from Task 2
arima_info = joblib.load('../data/processed/models/tsla_arima_info.joblib')
best_order = arima_info['order']
print(f'Best ARIMA order from Task 2: {best_order}')

Best ARIMA order from Task 2: (5, 1, 0)


## 3. Fit Final ARIMA Model on Full Data

For future forecasting, we refit the model using ALL available historical data.

In [4]:
print(f'Fitting ARIMA{best_order} on full TSLA history...')

final_model = ARIMA(tsla.values, order=best_order)
final_fit = final_model.fit()

print('Model fitted successfully!')
print(f'AIC: {final_fit.aic:.2f}')
print(f'BIC: {final_fit.bic:.2f}')

Fitting ARIMA(5, 1, 0) on full TSLA history...
Model fitted successfully!
AIC: -320.86
BIC: -285.29


## 4. Generate Future Forecasts (6 and 12 Months)

In [5]:
# Forecast horizons
HORIZON_6M = 126   # ~6 months of trading days
HORIZON_12M = 252  # ~12 months of trading days

# Use 12-month forecast for main analysis
FORECAST_HORIZON = HORIZON_12M

# Generate future business dates
last_date = tsla.index[-1]
future_dates = pd.bdate_range(start=last_date + pd.Timedelta(days=1), periods=FORECAST_HORIZON)

print(f'Forecast period: {future_dates[0].date()} to {future_dates[-1].date()}')
print(f'Forecast horizon: {FORECAST_HORIZON} trading days (~{FORECAST_HORIZON/21:.0f} months)')

Forecast period: 2026-01-15 to 2027-01-01
Forecast horizon: 252 trading days (~12 months)


In [ ]:
# Get forecast with confidence intervals
forecast_result = final_fit.get_forecast(steps=FORECAST_HORIZON)

forecast_mean = np.asarray(forecast_result.predicted_mean).reshape(-1)
forecast_ci = np.asarray(forecast_result.conf_int(alpha=0.05))  # 95% CI

# Ensure CI has two columns
if forecast_ci.ndim == 1:
    forecast_ci = np.column_stack([forecast_ci, forecast_ci])

lower_95 = forecast_ci[:, 0]
upper_95 = forecast_ci[:, 1]

# Align lengths defensively
n = min(len(forecast_mean), len(lower_95), len(upper_95), len(future_dates))
forecast_mean = forecast_mean[:n]
lower_95 = lower_95[:n]
upper_95 = upper_95[:n]
forecast_index = future_dates[:n]

# Create forecast DataFrame
forecast_df = pd.DataFrame({
    'Forecast': forecast_mean,
    'Lower_95': lower_95,
    'Upper_95': upper_95
}, index=forecast_index)

print('Forecast generated!')
print(f'\nFirst 5 days:')
print(forecast_df.head())
print(f'\nLast 5 days:')
print(forecast_df.tail())

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

In [ ]:
# Save forecast for Task 4
forecast_df.to_csv('../data/processed/tsla_forecast.csv')
print('Forecast saved to data/processed/tsla_forecast.csv')

## 5. Forecast Visualization with Confidence Intervals

In [ ]:
# Plot: Historical + Forecast with CI
fig, ax = plt.subplots(figsize=(16, 8))

# Historical data (last 2 years for clarity)
hist_start = tsla.index[-504:]  # ~2 years
ax.plot(tsla.loc[hist_start[0]:], label='Historical', color='black', linewidth=1.5)

# Forecast
ax.plot(forecast_df.index, forecast_df['Forecast'], label='Forecast', color='blue', linewidth=2)

# Confidence interval
ax.fill_between(
    forecast_df.index,
    forecast_df['Lower_95'],
    forecast_df['Upper_95'],
    color='blue', alpha=0.2, label='95% Confidence Interval'
)

# Mark last actual price
ax.axvline(x=last_date, color='red', linestyle='--', linewidth=1, label='Forecast Start')
ax.scatter([last_date], [tsla.iloc[-1]], color='red', s=100, zorder=5)

ax.set_title('TSLA Stock Price: Historical Data and 12-Month Forecast', fontsize=14, fontweight='bold')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Price ($)', fontsize=12)
ax.legend(loc='upper left', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../data/processed/tsla_forecast_ci.png', dpi=150, bbox_inches='tight')
plt.show()

print('Forecast visualization saved!')

## 6. Trend Analysis

In [ ]:
# Calculate key metrics
current_price = tsla.iloc[-1]
forecast_end_price = forecast_df['Forecast'].iloc[-1]
forecast_6m_price = forecast_df['Forecast'].iloc[HORIZON_6M-1] if len(forecast_df) >= HORIZON_6M else forecast_df['Forecast'].iloc[-1]

# Returns
return_6m = (forecast_6m_price / current_price - 1) * 100
return_12m = (forecast_end_price / current_price - 1) * 100

# CI width at different horizons
ci_width_1m = forecast_df['Upper_95'].iloc[21] - forecast_df['Lower_95'].iloc[21]
ci_width_6m = forecast_df['Upper_95'].iloc[HORIZON_6M-1] - forecast_df['Lower_95'].iloc[HORIZON_6M-1]
ci_width_12m = forecast_df['Upper_95'].iloc[-1] - forecast_df['Lower_95'].iloc[-1]

print('=' * 60)
print('FORECAST SUMMARY')
print('=' * 60)
print(f'Current Price:      ${current_price:.2f}')
print(f'6-Month Forecast:   ${forecast_6m_price:.2f} ({return_6m:+.1f}%)')
print(f'12-Month Forecast:  ${forecast_end_price:.2f} ({return_12m:+.1f}%)')
print()
print('Confidence Interval Width:')
print(f'  1 Month:  ${ci_width_1m:.2f}')
print(f'  6 Months: ${ci_width_6m:.2f}')
print(f'  12 Months: ${ci_width_12m:.2f}')

In [ ]:
# Determine trend direction
if return_12m > 5:
    trend = 'UPWARD'
elif return_12m < -5:
    trend = 'DOWNWARD'
else:
    trend = 'SIDEWAYS/STABLE'

print(f'\nOverall Trend Direction: {trend}')

### Trend Analysis Summary

*(This cell contains the written trend analysis - will be populated based on forecast results)*

In [ ]:
trend_analysis = f"""
TREND ANALYSIS (1-2 Paragraphs)
{'=' * 60}

The ARIMA{best_order} model forecasts Tesla's stock price over the next 12 months 
with a {trend.lower()} trajectory. Starting from the current price of ${current_price:.2f}, 
the model projects a 6-month target of ${forecast_6m_price:.2f} ({return_6m:+.1f}%) and 
a 12-month target of ${forecast_end_price:.2f} ({return_12m:+.1f}%). The forecast 
suggests {'continued momentum' if return_12m > 0 else 'potential headwinds'} for TSLA 
based on historical price patterns captured by the ARIMA model.

However, the confidence intervals widen significantly over the forecast horizon, 
reflecting increasing uncertainty. At 1 month, the 95% CI width is ${ci_width_1m:.2f}, 
expanding to ${ci_width_6m:.2f} at 6 months and ${ci_width_12m:.2f} at 12 months. 
This widening uncertainty is typical for financial time series and underscores that 
longer-term forecasts should be interpreted with caution. The model captures historical 
trends but cannot account for unforeseen market events, regulatory changes, or 
macroeconomic shifts that could significantly impact Tesla's stock price.
"""

print(trend_analysis)

## 7. Opportunities and Risks Assessment

In [ ]:
opportunities_risks = f"""
{'=' * 60}
MARKET OPPORTUNITIES AND RISKS
{'=' * 60}

OPPORTUNITIES:
{'─' * 40}
• {'Forecasted price appreciation of ' + f'{return_12m:.1f}% over 12 months suggests potential upside' if return_12m > 0 else 'Current valuation may present buying opportunity if fundamentals remain strong'}
• Short-term forecasts (1-3 months) have tighter confidence intervals, offering more reliable entry/exit signals
• TSLA's high volatility creates opportunities for active trading strategies
• Diversification benefits when combined with stable assets (BND) and broad market exposure (SPY)

RISKS:
{'─' * 40}
• Wide confidence intervals at 12 months (${ci_width_12m:.2f}) indicate high forecast uncertainty
• {'Downside risk: Lower 95% CI at 12 months is $' + f'{forecast_df["Lower_95"].iloc[-1]:.2f}' + f' ({((forecast_df["Lower_95"].iloc[-1]/current_price)-1)*100:.1f}% from current)'}
• ARIMA model assumes historical patterns persist; regime changes could invalidate forecasts
• External factors not captured: EV competition, regulatory changes, macroeconomic conditions
• High volatility means significant short-term price swings are likely

RELIABILITY ASSESSMENT BY HORIZON:
{'─' * 40}
• 1-3 Months:  MODERATE reliability - CI relatively tight, useful for tactical decisions
• 3-6 Months:  LOW-MODERATE reliability - CI widening, use with caution
• 6-12 Months: LOW reliability - CI very wide, treat as directional guidance only
"""

print(opportunities_risks)

## 8. Confidence Interval Analysis

In [ ]:
# Plot CI width over time
ci_width = forecast_df['Upper_95'] - forecast_df['Lower_95']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# CI width over time
ax1 = axes[0]
ax1.plot(forecast_df.index, ci_width, color='purple', linewidth=2)
ax1.set_title('95% Confidence Interval Width Over Forecast Horizon', fontsize=12, fontweight='bold')
ax1.set_xlabel('Date')
ax1.set_ylabel('CI Width ($)')
ax1.grid(True, alpha=0.3)

# CI as percentage of forecast
ax2 = axes[1]
ci_pct = (ci_width / forecast_df['Forecast']) * 100
ax2.plot(forecast_df.index, ci_pct, color='orange', linewidth=2)
ax2.set_title('CI Width as Percentage of Forecast Price', fontsize=12, fontweight='bold')
ax2.set_xlabel('Date')
ax2.set_ylabel('CI Width (%)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../data/processed/ci_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
reliability_assessment = """
CRITICAL ASSESSMENT OF FORECAST RELIABILITY
============================================

The reliability of time series forecasts degrades predictably with horizon length.
This is a fundamental property of forecasting models, not a weakness of ARIMA specifically.

Key observations:

1. SHORT-TERM (1-4 weeks): The confidence intervals are relatively narrow, and the 
   forecast closely follows recent price momentum. These predictions are most actionable
   for tactical portfolio adjustments.

2. MEDIUM-TERM (1-3 months): Uncertainty grows but remains manageable. Forecasts can
   inform quarterly rebalancing decisions, though with appropriate risk buffers.

3. LONG-TERM (6-12 months): The confidence intervals become very wide, often spanning
   a range that includes both significant gains and losses. At this horizon, the
   forecast should be treated as directional guidance rather than a precise target.

Implications for Portfolio Management:
- Use short-term forecasts for timing decisions
- Use long-term trend direction for strategic allocation
- Always incorporate the uncertainty (CI) into risk calculations
- Combine model forecasts with fundamental analysis and market intelligence
"""

print(reliability_assessment)

In [ ]:
# Save key metrics for Task 4
forecast_metrics = {
    'current_price': current_price,
    'forecast_6m_price': forecast_6m_price,
    'forecast_12m_price': forecast_end_price,
    'return_6m_pct': return_6m,
    'return_12m_pct': return_12m,
    'trend': trend
}
joblib.dump(forecast_metrics, '../data/processed/models/tsla_forecast_metrics.joblib')
print('Forecast metrics saved for Task 4!')

In [ ]:
print('\n' + '=' * 60)
print('TASK 3 COMPLETE')
print('=' * 60)
print('\nDeliverables:')
print('  ✓ Forecast visualization with confidence intervals')
print('  ✓ Trend analysis summary')
print('  ✓ List of identified opportunities and risks')
print('  ✓ Critical assessment of forecast reliability')
print('\nSaved outputs:')
print('  - data/processed/tsla_forecast.csv')
print('  - data/processed/tsla_forecast_ci.png')
print('  - data/processed/ci_analysis.png')
print('  - data/processed/models/tsla_forecast_metrics.joblib')